# OutputParser 练习

# StrOutputParser

In [2]:
# 导入所需的库
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser

# --- 1. 定义LLM模型 ---
# 这里我们使用Claude 3.5 Sonnet模型
# 注意：你需要先设置你的API密钥和可能的访问地址作为环境变量
import os
from langchain_deepseek import ChatDeepSeek
model = ChatDeepSeek(
    temperature=0,  # temperature=0表示输出更加确定性，不会随机性太强
    model_name="deepseek-chat",  # 模型名称
    api_key=os.getenv("DEEPSEEK_API_KEY")  # 从环境变量中获取API密钥
)

# --- 2. 定义一个工具 ---
# @tool装饰器可以将一个Python函数快速转换为Langchain可以使用的工具
@tool
def get_weather(location: str) -> str:
    """根据地名返回当地实时天气。"""
    # 这是一个示例工具，为了演示，它总是返回相同的结果
    return "天气晴朗22度"

# --- 3. 将工具绑定到LLM ---
# .bind_tools()方法让模型“知道”它可以使用这个工具
llm_with_tools = model.bind_tools([get_weather])

# --- 4. 构造一个链 (Chain) ---
# 使用管道符“|”将模型和输出解析器连接起来
# StrOutputParser可以简化输出，直接提取文本内容
chain = llm_with_tools | StrOutputParser()

# --- 5. 调用链并执行 ---
# .invoke()方法是运行一个链的标准方式
response = chain.invoke("北京市今天的天气如何？")

# --- 6. 打印结果 ---
print(response)

我来帮您查询北京市今天的天气情况。


# PydanticOutputParser

In [5]:
# --- 1. 引入依赖包 ---
# 这里的pydantic版本为v2
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, model_validator
from langchain_deepseek import ChatDeepSeek
import os

# --- 2. 使用deepseek模型 ---
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
)

# --- 3. 定义一个名为Joke的数据模型 ---
# 必须包含的数据字段：铺垫(setup)、抖包袱(punchline)
class Joke(BaseModel):
    setup: str = Field(description="笑话中的铺垫问题，必须以问号结尾")
    punchline: str = Field(description="笑话中回答铺垫问题的部分，通常是一种抖包袱方式回答铺垫问题，例如谐音、会错意等")

    # 验证器, 你可以根据自己的数据情况进行自定义
    # 注意mode=before意思是数据被转成pydantic模型的字段之前，对原始数据进行验证
    # @model_validator(mode="before")
    # @classmethod
    # def question_ends_with_question_mark(cls, values: dict) -> dict:
    #     setup = values.get("setup")
    #     # 如果铺垫问题没有以问号结尾则抛出错误
    #     if setup and setup[-1] != "?":
    #         raise ValueError("Badly formed question!")
    #     return values

# --- 4. 实例化解析器、提示词模板 ---
parser = PydanticOutputParser(pydantic_object=Joke)

# 注意，提示词模板中需要部分格式化解析器的格式要求 format_instructions
prompt = PromptTemplate(
    template="回答用户的查询。\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# --- 5. 使用LCEL语法组合一个简单的链 ---
prompt_and_model = prompt | llm
output = prompt_and_model.invoke({"query": "给我讲一个笑话"})

# 这里我们手动调用解析器来解析模型的输出
# (在下一个示例中会看到更简洁的写法)
parsed_output = parser.invoke(output)

print("--- 原始输出 ---")
print(output)
print("\n--- 解析后的输出 ---")
print(parsed_output)

# 你现在可以像操作一个对象一样操作解析后的结果
print("\n--- 单独访问字段 ---")
print("铺垫:", parsed_output.setup)
print("包袱:", parsed_output.punchline)

--- 原始输出 ---
content='{\n  "setup": "为什么鸡会过马路？",\n  "punchline": "为了到另一边去！"\n}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 229, 'total_tokens': 256, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 192}, 'prompt_cache_hit_tokens': 192, 'prompt_cache_miss_tokens': 37}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'a45aa4d9-6d9b-4da9-8013-fe835e5c2c32', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--66d0d7b9-f369-45ed-91f0-34462be892a0-0' usage_metadata={'input_tokens': 229, 'output_tokens': 27, 'total_tokens': 256, 'input_token_details': {'cache_read': 192}, 'output_token_details': {}}

--- 解析后的输出 ---
setup='为什么鸡会过马路？' punchline='为了到另一边去！'

--- 单独访问字段 ---
铺垫: 为什么鸡会过马路？
包袱: 为了到另一边去！


# JsonOutputParser + stream

In [7]:
# --- 1. 引入依赖包 ---
# 这里的pydantic版本为v2
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, model_validator
from langchain_deepseek import ChatDeepSeek
import os

# --- 2. 使用deepseek模型 ---
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
)

# --- 3. 定义一个名为Joke的数据模型 ---
# 必须包含的数据字段：铺垫(setup)、抖包袱(punchline)
class Joke(BaseModel):
    setup: str = Field(description="笑话中的铺垫问题，必须以问号结尾")
    punchline: str = Field(description="笑话中回答铺垫问题的部分，通常是一种抖包袱方式回答铺垫问题，例如谐音、会错意等")

    # 验证器, 你可以根据自己的数据情况进行自定义
    # 注意mode=before意思是数据被转成pydantic模型的字段之前，对原始数据进行验证
    # @model_validator(mode="before")
    # @classmethod
    # def question_ends_with_question_mark(cls, values: dict) -> dict:
    #     setup = values.get("setup")
    #     # 如果铺垫问题没有以问号结尾则抛出错误
    #     if setup and setup[-1] != "?":
    #         raise ValueError("Badly formed question!")
    #     return values

# 实例化解析器，提示词模板
# 注意：截图中高亮部分可能是视频中的笔误，应该是 PydanticOutputParser
parser = JsonOutputParser(pydantic_object=Joke)

# 注意，提示词模板中需要部分格式化解析器的格式要求 format_instructions
prompt = PromptTemplate(
    template="回答用户的查询。\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 使用LCEL语法组合一个完整的链
chain = prompt | llm | parser

# 方式一：一次性调用 (Invoke)
print("--- 使用 .invoke() ---")
output = chain.invoke({"query": "给我讲一个笑话"})
print(output)
print(type(output)) # 查看输出的类型

# # 这行代码在截图中被注释掉了，它的作用是单独查看格式化指令
# print(parser.get_format_instructions())

# 方式二：流式输出 (Stream)
print("\n--- 使用 .stream() ---")
# for s in chain.stream({"query":"给我讲一个关于程序员编程的笑话"}):
#     print(s)

# 为了更好地展示流式效果，我们只看模型部分的流式输出
streaming_chain = prompt | llm
for chunk in streaming_chain.stream({"query": "给我讲一个关于程序员编程的笑话"}):
    print(chunk.content, end="", flush=True)

--- 使用 .invoke() ---
{'setup': '为什么程序员总是分不清万圣节和圣诞节？', 'punchline': '因为 Oct 31 等于 Dec 25。'}
<class 'dict'>

--- 使用 .stream() ---
```json
{
  "setup": "为什么程序员总是分不清万圣节和圣诞节？",
  "punchline": "因为 Oct 31 等于 Dec 25"
}
```

#  JsonOutputParser

In [8]:
# --- 1. 引入依赖包 ---
# 这里的pydantic版本为v2
import os
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_deepseek import ChatDeepSeek

# --- 2. 使用deepseek模型 ---
# 假设 llm 已经定义好
llm = ChatDeepSeek(
    model="deepseek-chat", # 截图中为 "Pro/deepseek-ai/DeepSeek-V3"，请按需修改
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    # api_base=os.environ.get("DEEPSEEK_API_BASE"), # 如果使用代理，请取消此行注释
)

# --- 3. 定义用户的查询 ---
joke_query = "Tell me a joke."

# --- 4. 实例化解析器 ---
# 这次我们使用通用的JsonOutputParser
parser = JsonOutputParser()

# --- 5. 创建提示词模板 ---
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# --- 6. 使用LCEL语法组合一个完整的链 ---
chain = prompt | llm | parser

# --- 7. 调用链并打印结果 ---
result = chain.invoke({"query": joke_query})
print(result)

# 查看一下返回结果的类型
print(type(result))

{'joke': "Why don't scientists trust atoms? Because they make up everything."}
<class 'dict'>


# XMLOutputParser

In [9]:
# --- 1. 引入依赖包 ---
import os
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_deepseek import ChatDeepSeek

# --- 2. 使用deepseek模型 ---
llm = ChatDeepSeek(
    model="deepseek-chat", # 截图中为 "Pro/deepseek-ai/DeepSeek-V3"，请按需修改
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
)

# --- 3. 定义用户的查询 ---
actor_query = "Generate the shortened filmography for Tom Hanks."

# --- 4. 实例化XML解析器 ---
parser = XMLOutputParser()

# (可选) 打印出格式化指令，看看它会告诉LLM什么
# print(parser.get_format_instructions())

# --- 5. 创建提示词模板 ---
prompt = PromptTemplate(
    template="""{query}\n{format_instructions}""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# --- 6. 使用LCEL语法组合一个完整的链 ---
chain = prompt | llm | parser

# --- 7. 调用链并打印结果 ---
output = chain.invoke({"query": actor_query})
print(output)

{'filmography': [{'actor': [{'name': 'Tom Hanks'}, {'movies': [{'movie': [{'title': 'Splash'}, {'year': '1984'}]}, {'movie': [{'title': 'Big'}, {'year': '1988'}]}, {'movie': [{'title': 'A League of Their Own'}, {'year': '1992'}]}, {'movie': [{'title': 'Philadelphia'}, {'year': '1993'}]}, {'movie': [{'title': 'Forrest Gump'}, {'year': '1994'}]}, {'movie': [{'title': 'Apollo 13'}, {'year': '1995'}]}, {'movie': [{'title': 'Saving Private Ryan'}, {'year': '1998'}]}, {'movie': [{'title': 'Cast Away'}, {'year': '2000'}]}, {'movie': [{'title': 'The Da Vinci Code'}, {'year': '2006'}]}, {'movie': [{'title': 'Captain Phillips'}, {'year': '2013'}]}, {'movie': [{'title': 'Sully'}, {'year': '2016'}]}, {'movie': [{'title': 'A Beautiful Day in the Neighborhood'}, {'year': '2019'}]}, {'movie': [{'title': 'Greyhound'}, {'year': '2020'}]}]}]}]}
